In [4]:
from google.colab import userdata
openrouter_api_key = userdata.get('openrouter')

In [3]:
%%capture
!pip install -q langgraph langchain langchain-openai
!pip install langgraph langchain-openai langchain-anthropic langchain-community duckduckgo-search arxiv chromadb pydantic
!pip install langgraph-checkpoint-sqlite
!pip install sentence-transformers

In [5]:
# ---------- Standard Python ----------
import asyncio
import json
import logging
import hashlib
import uuid
from datetime import datetime
from enum import Enum
from dataclasses import dataclass, field
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional, TypedDict, Annotated

import nest_asyncio
nest_asyncio.apply()


# ---------- Pydantic ----------
from pydantic import BaseModel, Field, validator


# ---------- LangGraph ----------
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver


# ---------- LLM Providers ----------
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic


# ---------- Tools / Search ----------
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import ArxivAPIWrapper


# ---------- Document Loading ----------
from langchain_community.document_loaders import WebBaseLoader


# ---------- Text Splitters (FIXED) ----------
from langchain_text_splitters import RecursiveCharacterTextSplitter


# ---------- Vector Stores ----------
from langchain_community.vectorstores import Chroma


# ---------- Chains ----------
# from langchain.chains import RetrievalQA


# ---------- Prompts & Messages ----------
from langchain_classic.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


# ---------- Runnables ----------
from langchain_core.runnables import RunnableLambda, RunnableConfig


# ---------- Callbacks ----------
# from langchain_core.callbacks import BaseCallbackHandler
# from langchain_community.callbacks import FileCallbackHandler

from langchain_community.embeddings import HuggingFaceEmbeddings


In [6]:
class ResearchTopic(BaseModel):
    title: str = Field(..., min_length=5, max_length=200)
    domain: str = Field(..., description="e.g., 'AI', 'Physics', 'Biology'")
    complexity: str = Field("intermediate", pattern="^(beginner|intermediate|advanced|expert)$")
    subtopics: List[str] = Field(default_factory=list)

    @validator('title')
    def title_must_be_meaningful(cls, v):
        if len(v.split()) < 2:
            raise ValueError("Title must be at least 2 words")
        return v

/tmp/ipython-input-3115356385.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('title')


In [7]:
class ResearchState(TypedDict):
    messages: Annotated[List[Any], add_messages]
    topic: ResearchTopic
    research_phase: str
    search_results: Dict[str, Any]
    sources: List[Dict[str, str]]
    analysis: Dict[str, Any]
    literature_review: str
    methodology: str
    findings: List[Dict[str, Any]]
    limitations: List[str]
    recommendations: List[str]
    citations: List[Dict[str, str]]
    validation_errors: List[str]
    metadata: Dict[str, Any]
    agent_logs: List[Dict[str, Any]]


In [8]:
class AgentType(Enum):
    """Types of specialized agents"""
    RESEARCH_COORDINATOR = "research_coordinator"
    SEARCH_SPECIALIST = "search_specialist"
    ANALYST = "analyst"
    VALIDATOR = "validator"
    SYNTHESIZER = "synthesizer"
    CRITIC = "critic"
    WRITER = "writer"

In [9]:
@dataclass
class AgentConfig:
    """Configuration for individual agents"""
    agent_type: AgentType
    llm_model: str = "liquid/lfm-2.5-1.2b-instruct:free"
    temperature: float = 0.3
    max_tokens: int = 2000
    tools: List[Any] = field(default_factory=list)
    system_prompt: str = ""
    is_async: bool = False

In [10]:
class AdvancedResearch:
  def __init__(self):
    self.search_tool = DuckDuckGoSearchRun()
    self.arxiv_wrapper = ArxivAPIWrapper()
    self.cache = {}

  def search_with_cache(self,query:str,max_results:int=5) -> Dict[str,any]:
    cache_key = hashlib.md5(query.encode()).hexdigest() # Corrected from m5 to md5
    if cache_key in self.cache:
      return self.cache[cache_key]

    try:
      with ThreadPoolExecutor(max_workers=3) as executor: # Corrected typo: ThreadPoolEcecutor -> ThreadPoolExecutor
        futures=[
            executor.submit(self._web_search,query,max_results),
            executor.submit(self._arxiv_search,query,max_results),
            executor.submit(self._scholar_search,query,max_results),
        ]
        results=[]
        for future in as_completed(futures):
          try:
            results.extend(future.result())
          except Exception as e:
            print(f"Search error {e}")
      unique_results = self._deduplicate_results(results)
      result_dict={
          "query":query,
          "timestamp":datetime.now().isoformat(),
          "results":unique_results[:max_results]
      }
      self.cache[cache_key]=result_dict
      return result_dict
    except Exception as e: # Added missing except block
      print(f"An error occurred in search_with_cache: {e}")
      return {
          "query": query,
          "timestamp": datetime.now().isoformat(),
          "results": [],
          "error": str(e)
      }

  def _web_search(self,query:str,max_results:int) -> List[Dict]:
    try:
      results = self.search_tool.run(query)
      return [{
          "source":"web",
          "content":results[:1000],
          "relevance_score":0.8
      }]
    except:
      return []

  def _arxiv_search(self, query: str, max_results: int) -> List[Dict]:
        """Search arXiv for academic papers"""
        try:
            docs = self.arxiv_wrapper.load(query[:300])
            return [{
                "source": "arxiv",
                "title": doc.metadata.get("Title", ""),
                "authors": doc.metadata.get("Authors", []),
                "summary": doc.page_content[:500],
                "published": doc.metadata.get("Published", ""),
                "relevance_score": 0.9
            } for doc in docs[:max_results]]
        except:
            return []

  def _scholar_search(self, query: str, max_results: int) -> List[Dict]: # Corrected indentation
        """Mock scholar search (replace with actual API)"""
        return [{
            "source": "scholar",
            "title": f"Research on {query}",
            "abstract": f"This paper discusses {query} in detail...",
            "citations": 42,
            "relevance_score": 0.85
        }]

  def _deduplicate_results(self, results: List[Dict]) -> List[Dict]:
    seen = set()
    unique=[]
    for result in results:
      content_hash = hashlib.md5(str(result.get('title','') + result.get('content', '')).encode()).hexdigest()
      if content_hash not in seen:
        seen.add(content_hash)
        unique.append(result) # Corrected: append result, not hash
    return unique

In [11]:
class ContentAnalyzer:
  def __init__(self):
    self.Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    self.text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap=200
    )
  def analyze_content(self,content) -> Dict[str,any]:
    chunks = self.text_splitter.split_text(content)
    embeddings = self.Embeddings.embed_documents(chunks)
    word_count = len(content.split())
    sent_count =len(content.split('.'))
    return {
        "chunks": len(chunks),
            "word_count": word_count,
            "sentence_count": sent_count
    }

In [12]:
class BaseAgent:
  def __init__(self,config = AgentConfig):
    self.config = config
    self.llm = self.initiate_llm()
    self.tools = config.tools
    self.logger = self._setup_logger()
  def initiate_llm(self):
    return ChatOpenAI(
    openai_api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
    model="liquid/lfm-2.5-1.2b-instruct:free",
)
  def _setup_logger(self):
        """Setup agent-specific logger"""
        logger = logging.getLogger(self.config.agent_type.value)
        logger.setLevel(logging.INFO)
        handler = logging.FileHandler(f"logs/{self.config.agent_type.value}.log")
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

  def log_activity(self, activity: str, metadata: Dict = None):
        """Log agent activity"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "agent": self.config.agent_type.value,
            "activity": activity,
            "metadata": metadata or {}
        }
        self.logger.info(json.dumps(log_entry))
        return log_entry

In [13]:
class ResearchCoordinatorAgent(BaseAgent):
    """Orchestrates the research process"""
    def __init__(self):
        config = AgentConfig(
            agent_type=AgentType.RESEARCH_COORDINATOR,
            system_prompt="""You are a Research Coordinator. Your responsibilities:
            1. Analyze the research topic and break it into subtopics
            2. Determine the appropriate research methodology
            3. Coordinate between different specialist agents
            4. Monitor research progress and quality
            5. Synthesize final findings

            You must ensure comprehensive coverage and academic rigor."""
        )
        super().__init__(config)

    async def coordinate(self, state: ResearchState) -> Dict[str, Any]:
        """Coordinate the research process"""
        self.log_activity("coordination_started", {"topic": state["topic"].dict()})

        # Analyze topic and create research plan
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.config.system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            HumanMessage(content=f"""
            Research Topic: {state['topic'].title}
            Domain: {state['topic'].domain}
            Complexity: {state['topic'].complexity}

            Create a comprehensive research plan including:
            1. Key research questions
            2. Required data sources
            3. Methodology outline
            4. Success criteria
            5. Timeline estimate
            """)
        ])

        chain = prompt | self.llm
        response = await chain.ainvoke({
            "messages": state.get("messages", [])
        })

        # Parse the response and update state
        coordination_result = {
            "research_plan": response.content,
            "next_phase": "literature_review",
            "assigned_agents": [
                AgentType.SEARCH_SPECIALIST.value,
                AgentType.ANALYST.value
            ],
            "timestamp": datetime.now().isoformat()
        }

        self.log_activity("coordination_completed", coordination_result)

        return {"research_phase": "literature_review", "analysis": coordination_result}

class SearchSpecialistAgent(BaseAgent):
    """Specializes in finding and evaluating sources"""
    def __init__(self):
        config = AgentConfig(
            agent_type=AgentType.SEARCH_SPECIALIST,
            system_prompt="""You are a Search Specialist. Your responsibilities:
            1. Find relevant academic and web sources
            2. Evaluate source credibility and relevance
            3. Extract key information from sources
            4. Maintain source citations
            5. Identify knowledge gaps

            Prioritize recent, authoritative sources."""
        )
        super().__init__(config)
        self.search_tool = AdvancedResearch()
        self.content_analyzer = ContentAnalyzer()

    async def search(self, state: ResearchState) -> Dict[str, Any]:
        """Perform comprehensive search"""
        topic = state["topic"]

        # Generate search queries based on topic
        queries = [
            f"{topic.title} recent developments",
            f"{topic.domain} {topic.title} research papers",
            f"{topic.title} methodology best practices"
        ]

        # Execute parallel searches
        search_tasks = [self._execute_search(query) for query in queries]
        search_results = await asyncio.gather(*search_tasks)

        # Process and analyze results
        processed_results = []
        citations = []

        for result_batch in search_results:
            for result in result_batch.get("results", []):
                # Analyze content quality
                if "content" in result or "summary" in result:
                    content = result.get("content", result.get("summary", ""))
                    analysis = self.content_analyzer.analyze_content(content)

                    processed_result = {
                        **result,
                        "analysis": analysis,
                        "processed_at": datetime.now().isoformat()
                    }
                    processed_results.append(processed_result)

                    # Create citation
                    citation = {
                        "id": str(uuid.uuid4()),
                        "title": result.get("title", "Untitled"),
                        "authors": result.get("authors", ["Unknown"]),
                        "source": result.get("source", "Unknown"),
                        "url": result.get("url", ""),
                        "accessed_at": datetime.now().isoformat()
                    }
                    citations.append(citation)

        self.log_activity("search_completed", {
            "queries": queries,
            "results_found": len(processed_results),
            "citations_generated": len(citations)
        })

        return {
            "search_results": processed_results,
            "sources": processed_results,
            "citations": citations,
            "research_phase": "analysis"
        }

    async def _execute_search(self, query: str) -> Dict[str, Any]:
        """Execute a single search query"""
        return await asyncio.get_event_loop().run_in_executor(
            None, self.search_tool.search_with_cache, query, 3
        )

class AnalystAgent(BaseAgent):
    """Analyzes and synthesizes information"""
    def __init__(self):
        config = AgentConfig(
            agent_type=AgentType.ANALYST,
            system_prompt="""You are an Analysis Specialist. Your responsibilities:
            1. Analyze collected data and sources
            2. Identify patterns, trends, and relationships
            3. Evaluate evidence quality and bias
            4. Generate insights and hypotheses
            5. Create structured analysis reports

            Maintain objectivity and academic rigor."""
        )
        super().__init__(config)

    async def analyze(self, state: ResearchState) -> Dict[str, Any]:
        """Analyze search results and generate insights"""
        search_results = state.get("search_results", [])
        topic = state["topic"]

        # Prepare data for analysis
        analysis_data = {
            "topic": topic.dict(),
            "sources_count": len(search_results),
            "sources_by_type": {},
            "key_findings": [],
            "methodologies_identified": [],
            "knowledge_gaps": []
        }

        # Categorize sources
        for result in search_results:
            source_type = result.get("source", "unknown")
            analysis_data["sources_by_type"][source_type] = \
                analysis_data["sources_by_type"].get(source_type, 0) + 1

        # Generate analysis using LLM
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.config.system_prompt),
            HumanMessage(content=f"""
            Analyze these research findings for topic: {topic.title}

            Search Results Summary:
            {json.dumps(search_results[:5], indent=2)}

            Provide a comprehensive analysis covering:
            1. Key trends and patterns
            2. Conflicting viewpoints
            3. Research methodologies used
            4. Evidence strength assessment
            5. Identified knowledge gaps
            6. Preliminary conclusions
            """)
        ])

        chain = prompt | self.llm
        response = await chain.ainvoke({})

        # Parse and structure analysis
        analysis_result = {
            "comprehensive_analysis": response.content,
            "metadata": analysis_data,
            "analysis_timestamp": datetime.now().isoformat(),
            "confidence_score": self._calculate_confidence(search_results)
        }

        self.log_activity("analysis_completed", {
            "topic": topic.title,
            "sources_analyzed": len(search_results),
            "confidence": analysis_result["confidence_score"]
        })

        return {
            "analysis": analysis_result,
            "research_phase": "synthesis",
            "findings": self._extract_findings(response.content)
        }

    def _calculate_confidence(self, results: List[Dict]) -> float:
        """Calculate confidence score based on source quality"""
        if not results:
            return 0.0

        scores = []
        for result in results:
            # Score based on source type and relevance
            base_score = result.get("relevance_score", 0.5)
            source_bonus = {
                "arxiv": 0.3,
                "scholar": 0.3,
                "web": 0.1
            }.get(result.get("source", ""), 0.0)
            scores.append(min(1.0, base_score + source_bonus))

        return sum(scores) / len(scores) if scores else 0.0

    def _extract_findings(self, analysis: str) -> List[Dict[str, Any]]:
        """Extract structured findings from analysis"""
        # This is a simplified version - in production, use more sophisticated extraction
        findings = []
        lines = analysis.split('\n')

        for line in lines:
            if any(keyword in line.lower() for keyword in ['finding', 'conclusion', 'result', 'shows']):
                findings.append({
                    "id": str(uuid.uuid4()),
                    "content": line.strip(),
                    "category": self._categorize_finding(line),
                    "confidence": 0.8
                })

        return findings

    def _categorize_finding(self, finding: str) -> str:
        """Categorize finding type"""
        finding_lower = finding.lower()
        if any(word in finding_lower for word in ['method', 'approach', 'technique']):
            return "methodology"
        elif any(word in finding_lower for word in ['result', 'finding', 'shows', 'indicates']):
            return "result"
        elif any(word in finding_lower for word in ['limitation', 'gap', 'challenge']):
            return "limitation"
        elif any(word in finding_lower for word in ['recommend', 'suggest', 'propose']):
            return "recommendation"
        return "observation"

In [14]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver


In [20]:
import aiosqlite # Import aiosqlite for async SQLite connections

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.checkpoint.base import BaseCheckpointSaver
from langgraph.graph import StateGraph, END
from datetime import datetime
from typing import Dict, Any, List, Optional
import uuid
from langchain_core.messages import SystemMessage, HumanMessage

# Assuming ResearchState is defined elsewhere and necessary imports for agents

class ResearchAssistantGraph:
    """Main research assistant graph with complex workflow"""

    def __init__(self):
        # Correctly instantiate AsyncSqliteSaver with an aiosqlite connection
        self.memory = AsyncSqliteSaver(conn=aiosqlite.connect("research_checkpoints.db"))
        self.agents = self._initialize_agents()
        self.graph = self._build_graph()
        self.app = self._compile_graph()

    def _initialize_agents(self) -> Dict[str, Any]:  # Changed from AgentType to str
        """Initialize all specialized agents"""
        # Assuming these agent classes are defined elsewhere
        return {
            "research_coordinator": ResearchCoordinatorAgent(),
            "search_specialist": SearchSpecialistAgent(),
            "analyst": AnalystAgent(),
            # Additional agents can be added here
        }

    def _build_graph(self) -> StateGraph:
        """Build the complex research workflow graph"""
        workflow = StateGraph(ResearchState)

        # Add nodes for each agent
        workflow.add_node("research_coordinator", self._create_agent_node("research_coordinator"))
        workflow.add_node("search_specialist", self._create_agent_node("search_specialist"))
        workflow.add_node("analyst", self._create_agent_node("analyst"))
        workflow.add_node("validator", self._validator_node)
        workflow.add_node("synthesizer", self._synthesizer_node)
        workflow.add_node("writer", self._writer_node)

        # Define the workflow edges with conditional routing
        workflow.set_entry_point("research_coordinator")

        # Main research flow
        workflow.add_edge("research_coordinator", "search_specialist")
        workflow.add_edge("search_specialist", "analyst")

        # Conditional routing based on analysis quality
        workflow.add_conditional_edges(
            "analyst",
            self._route_based_on_quality,
            {
                "continue": "validator",
                "redo_search": "search_specialist",
                "escalate": "research_coordinator"
            }
        )

        # Conditional routing based on validation results from validator
        workflow.add_conditional_edges(
            "validator",
            self._route_from_validator,
            {
                "analyst": "analyst",       # If needs revision
                "synthesizer": "synthesizer" # If validation passed
            }
        )

        workflow.add_edge("synthesizer", "writer")
        workflow.add_edge("writer", END)

        return workflow

    def _create_agent_node(self, agent_type: str):
        """Create a node for a specific agent type"""
        agent = self.agents[agent_type]

        async def agent_node(state: ResearchState):
            if agent_type == "research_coordinator":
                return await agent.coordinate(state)
            elif agent_type == "search_specialist":
                return await agent.search(state)
            elif agent_type == "analyst":
                return await agent.analyze(state)
            else:
                return {}

        return agent_node

    async def _validator_node(self, state: ResearchState) -> Dict[str, Any]:
        """Validate research quality and completeness"""
        analysis = state.get("analysis", {})
        findings = state.get("findings", [])

        validation_results = {
            "timestamp": datetime.now().isoformat(),
            "checks": [],
            "passed": True,
            "issues": []
        }

        # Check 1: Sufficient sources
        sources_count = len(state.get("sources", []))
        if sources_count < 3:
            validation_results["checks"].append({
                "check": "minimum_sources",
                "passed": False,
                "message": f"Only {sources_count} sources found (minimum: 3)"
            })
            validation_results["passed"] = False
            validation_results["issues"].append("insufficient_sources")

        # Check 2: Analysis depth
        analysis_text = analysis.get("comprehensive_analysis", "")
        if len(analysis_text.split()) < 200:
            validation_results["checks"].append({
                "check": "analysis_depth",
                "passed": False,
                "message": f"Analysis too brief: {len(analysis_text.split())} words"
            })
            validation_results["passed"] = False
            validation_results["issues"].append("shallow_analysis")

        # Check 3: Findings clarity
        if len(findings) < 2:
            validation_results["checks"].append({
                "check": "findings_count",
                "passed": False,
                "message": f"Only {len(findings)} findings identified"
            })
            validation_results["passed"] = False
            validation_results["issues"].append("insufficient_findings")

        # Add passed checks
        validation_results["checks"].extend([
            {
                "check": "topic_relevance",
                "passed": True,
                "message": "All findings are relevant to research topic"
            },
            {
                "check": "citation_presence",
                "passed": True,
                "message": f"Found {len(state.get('citations', []))} citations"
            }
        ])

        return {
            "validation_errors": validation_results.get("issues", []),
            "validation_results": validation_results,
            "research_phase": "synthesis" if validation_results["passed"] else "needs_revision"
        }

    def _route_based_on_quality(self, state: ResearchState) -> str:
        """Route based on analysis quality"""
        analysis = state.get("analysis", {})
        confidence = analysis.get("confidence_score", 0.0)

        if confidence < 0.4:
            return "redo_search"
        elif confidence < 0.7:
            return "escalate"
        else:
            return "continue"

    def _route_from_validator(self, state: ResearchState) -> str:
        """Route based on validation results"""
        if state.get("research_phase") == "needs_revision":
            return "analyst"
        return "synthesizer"

    async def _synthesizer_node(self, state: ResearchState) -> Dict[str, Any]:
        """Synthesize final research report"""
        synthesis = {
            "executive_summary": self._generate_executive_summary(state),
            "key_findings": state.get("findings", []),
            "methodology": self._extract_methodology(state),
            "limitations": self._identify_limitations(state),
            "recommendations": self._generate_recommendations(state),
            "synthesized_at": datetime.now().isoformat(),
            "total_sources": len(state.get("sources", [])),
            "validation_status": state.get("validation_results", {}).get("passed", False)
        }

        return {
            "literature_review": self._create_literature_review(state),
            "methodology": synthesis["methodology"],
            "limitations": synthesis["limitations"],
            "recommendations": synthesis["recommendations"],
            "research_phase": "writing"
        }

    async def _writer_node(self, state: ResearchState) -> Dict[str, Any]:
        """Generate final research paper"""
        paper_sections = [
            "Title and Abstract",
            "Introduction",
            "Literature Review",
            "Methodology",
            "Findings and Analysis",
            "Discussion",
            "Limitations",
            "Conclusion and Recommendations",
            "References"
        ]

        final_paper = {
            "title": f"Research Report: {state['topic'].title}",
            "sections": paper_sections,
            "content": self._compile_final_paper(state),
            "generated_at": datetime.now().isoformat(),
            "word_count": sum(len(section.split()) for section in paper_sections),
            "citations_count": len(state.get("citations", [])),
            "metadata": state.get("metadata", {})
        }

        return {
            "final_paper": final_paper,
            "research_phase": "completed",
            "completion_time": datetime.now().isoformat()
        }

    def _compile_graph(self):
        """Compile the graph with memory and parallel processing"""
        # FIXED: Correct way to pass the checkpointer
        return self.graph.compile(
            checkpointer=self.memory,
            debug=False
        )

    # Helper methods for synthesis and writing (keep these as is)
    def _generate_executive_summary(self, state: ResearchState) -> str:
        """Generate executive summary"""
        topic = state["topic"]
        findings = state.get("findings", [])

        return f"""
        Executive Summary: {topic.title}

        This research investigated {topic.title} in the domain of {topic.domain}.
        Key findings include {len(findings)} major insights covering methodology,
        results and recommendations. The research demonstrates {'strong' if len(findings) > 3 else 'preliminary'}
        evidence supporting the initial research questions.
        """

    def _create_literature_review(self, state: ResearchState) -> str:
        """Create comprehensive literature review"""
        sources = state.get("sources", [])

        review = f"Literature Review\n\n"
        review += f"Based on {len(sources)} sources, the literature reveals:\n\n"

        for i, source in enumerate(sources[:10], 1):
            review += f"{i}. {source.get('title', 'Untitled')} "
            review += f"({source.get('source', 'Unknown source')})\n"
            review += f"   Key contribution: {source.get('content', '')[:200]}...\n\n"

        return review

    def _extract_methodology(self, state: ResearchState) -> str:
        """Extract methodology from analysis"""
        analysis = state.get("analysis", {})
        methods = analysis.get("metadata", {}).get("methodologies_identified", [])

        if methods:
            return "Methodologies identified: " + ", ".join(methods)
        return "Standard literature review and analysis methodology"

    def _identify_limitations(self, state: ResearchState) -> List[str]:
        """Identify research limitations"""
        limitations = []

        if len(state.get("sources", [])) < 5:
            limitations.append("Limited number of sources")

        if state.get("validation_results", {}).get("issues"):
            limitations.extend(state["validation_results"]["issues"])

        return limitations if limitations else ["Standard limitations of literature review methodology"]

    def _generate_recommendations(self, state: ResearchState) -> List[str]:
        """Generate research recommendations"""
        topic = state["topic"]

        return [
            f"Further empirical research on {topic.title}",
            "Longitudinal studies to validate findings",
            "Cross-disciplinary approaches for comprehensive understanding",
            "Application of findings in practical contexts"
        ]

    def _compile_final_paper(self, state: ResearchState) -> Dict[str, str]:
        """Compile final research paper sections"""
        return {
            "abstract": self._generate_executive_summary(state),
            "introduction": f"Introduction to {state['topic'].title} research...",
            "literature_review": state.get("literature_review", ""),
            "methodology": state.get("methodology", ""),
            "findings": "\n".join([f["content"] for f in state.get("findings", [])]),
            "discussion": "Analysis and interpretation of findings...",
            "limitations": "\n".join(state.get("limitations", [])),
            "conclusion": "Summary of research and implications...",
            "references": self._format_citations(state.get("citations", []))
        }

    def _format_citations(self, citations: List[Dict]) -> str:
        """Format citations in APA style"""
        formatted = []
        for i, citation in enumerate(citations, 1):
            formatted.append(
                f"{i}. {citation.get('authors', ['Author'])[0]} et al. "
                f"({citation.get('published', 'n.d.').split('-')[0]}). "
                f"{citation.get('title', 'Untitled')}. "
                f"{citation.get('source', 'Unknown')}."
            )
        return "\n".join(formatted)

    async def run_research(self, topic: ResearchTopic, config: Optional[Dict] = None) -> Dict[str, Any]:
        """Execute the complete research workflow"""
        # Initialize state
        initial_state = {
            "topic": topic,
            "messages": [
                SystemMessage(content="You are an advanced research assistant."),
                HumanMessage(content=f"Research topic: {topic.title}")
            ],
            "research_phase": "initiated",
            "search_results": {},
            "sources": [],
            "analysis": {},
            "literature_review": "",
            "methodology": "",
            "findings": [],
            "limitations": [],
            "recommendations": [],
            "citations": [],
            "validation_errors": [],
            "metadata": {
                "start_time": datetime.now().isoformat(),
                "topic_id": str(uuid.uuid4()),
                "config": config or {}
            },
            "agent_logs": []
        }

        # Execute the research workflow
        final_state = {}

        # FIXED: Proper way to stream with checkpointer
        config_dict = {
            "configurable": {
                "thread_id": str(uuid.uuid4()),  # Use unique thread_id
                "checkpointer": self.memory
            }
        }

        async for step in self.app.astream(initial_state, config=config_dict):
            for key, value in step.items():
                if key != "__end__":
                    print(f"Step: {key}")
                    final_state.update(value)

        return final_state

In [34]:
def display_research_improved(result):
    """Display research output with better formatting and data extraction"""

    print("\n" + "="*100)
    print("🎯 ADVANCED RESEARCH REPORT")
    print("="*100)

    # Extract topic info better
    topic_info = {}
    if isinstance(result.get('topic'), dict):
        topic_info = result['topic']
    elif hasattr(result.get('topic'), 'dict'):  # Pydantic model
        topic_info = result['topic'].dict()
    elif hasattr(result.get('topic'), 'model_dump'):  # Pydantic v2
        topic_info = result['topic'].model_dump()

    # 1. RESEARCH OVERVIEW
    print(f"\n📋 RESEARCH OVERVIEW")
    print(f"   {'─'*40}")
    if topic_info:
        print(f"   🔹 Topic: {topic_info.get('title', 'N/A')}")
        print(f"   🔹 Domain: {topic_info.get('domain', 'N/A')}")
        print(f"   🔹 Complexity: {topic_info.get('complexity', 'N/A')}")
        if topic_info.get('subtopics'):
            print(f"   🔹 Subtopics: {', '.join(topic_info['subtopics'])}")
    else:
        print(f"   Topic data not found in result")

    print(f"   🔹 Research Phase: {result.get('research_phase', 'N/A')}")
    print(f"   🔹 Sources Analyzed: {len(result.get('sources', []))}")
    print(f"   🔹 Key Findings: {len(result.get('findings', []))}")

    # 2. RESEARCH PLAN (from coordinator logs)
    print(f"\n📝 RESEARCH PLAN")
    print(f"   {'─'*40}")

    # Try to find the research plan from coordinator metadata
    research_plan = None
    if 'metadata' in result and 'research_plan' in result['metadata']:
        research_plan = result['metadata']['research_plan']
    elif 'analysis' in result and 'comprehensive_analysis' in result['analysis']:
        research_plan = result['analysis']['comprehensive_analysis']

    if research_plan:
        # Extract first few lines
        lines = research_plan.split('\n')[:10]
        for line in lines:
            if line.strip():
                print(f"   {line}")
        if len(research_plan.split('\n')) > 10:
            print(f"   ... (truncated, full plan: {len(research_plan.split())} words)")
    else:
        print("   No detailed research plan found")

    # 3. SOURCES with better formatting
    sources = result.get('sources', [])
    print(f"\n📚 SOURCES FOUND ({len(sources)})")
    print(f"   {'─'*40}")

    for i, source in enumerate(sources, 1):
        # Extract title
        title = source.get('title', 'Untitled')
        if title == 'Untitled' and 'content' in source:
            # Try to extract title from content
            content = source.get('content', '')
            if content and len(content.split()) > 2:
                title = ' '.join(content.split()[:5]) + '...'

        source_type = source.get('source', 'unknown').upper()
        relevance = source.get('relevance_score', 0)

        print(f"   {i}. [{source_type}] {title}")

        # Show snippet
        if 'content' in source and source['content']:
            content = source['content']
            snippet = content[:150].replace('\n', ' ')
            if len(content) > 150:
                snippet += '...'
            print(f"      └─ {snippet}")

        # Show metadata if available
        meta_info = []
        if 'published' in source:
            meta_info.append(f"Published: {source['published']}")
        if 'authors' in source and source['authors']:
            authors = source['authors'][:2]
            meta_info.append(f"Authors: {', '.join(authors)}" +
                           (f" + {len(source['authors'])-2} more" if len(source['authors']) > 2 else ""))
        if relevance:
            meta_info.append(f"Relevance: {relevance:.2f}")

        if meta_info:
            print(f"      └─ {' | '.join(meta_info)}")

    # 4. KEY FINDINGS - improved formatting
    findings = result.get('findings', [])
    print(f"\n🔍 KEY FINDINGS ({len(findings)})")
    print(f"   {'─'*40}")

    for i, finding in enumerate(findings, 1):
        content = finding.get('content', '').strip()
        category = finding.get('category', 'observation').upper()
        confidence = finding.get('confidence', 0)

        # Clean up the content
        if content.startswith('#') or content.startswith('##'):
            content = content.lstrip('#').strip()

        print(f"   {i}. [{category}]")
        print(f"      {content}")

        if confidence:
            confidence_bar = '█' * int(confidence * 10) + '░' * (10 - int(confidence * 10))
            print(f"      Confidence: {confidence:.2f} [{confidence_bar}]")
        print()

    # 5. ANALYSIS INSIGHTS
    if 'analysis' in result:
        analysis = result['analysis']
        print(f"\n🧠 ANALYSIS INSIGHTS")
        print(f"   {'─'*40}")

        if 'comprehensive_analysis' in analysis:
            insights = analysis['comprehensive_analysis']
            # Extract key points
            lines = insights.split('\n')
            for line in lines[:8]:  # Show first 8 lines
                if line.strip() and len(line.strip()) > 20:
                    print(f"   • {line.strip()}")

        if 'confidence_score' in analysis:
            print(f"   🔹 Overall Confidence Score: {analysis['confidence_score']:.2f}/1.0")

    # 6. LIMITATIONS & RECOMMENDATIONS
    limitations = result.get('limitations', [])
    if limitations:
        print(f"\n⚠️ IDENTIFIED LIMITATIONS")
        print(f"   {'─'*40}")
        for i, limitation in enumerate(limitations, 1):
            print(f"   {i}. {limitation}")

    recommendations = result.get('recommendations', [])
    if recommendations:
        print(f"\n💡 RESEARCH RECOMMENDATIONS")
        print(f"   {'─'*40}")
        for i, recommendation in enumerate(recommendations, 1):
            print(f"   {i}. {recommendation}")

    # 7. FINAL PAPER SECTIONS
    if 'final_paper' in result:
        paper = result['final_paper']
        print(f"\n📄 FINAL RESEARCH PAPER")
        print(f"   {'─'*40}")
        print(f"   Title: {paper.get('title', 'Research Report')}")
        print(f"   Status: ✓ COMPLETED")
        print(f"   Word Count: {paper.get('word_count', 0):,}")
        print(f"   Citations: {paper.get('citations_count', 0)}")

        # Show paper structure
        if 'sections' in paper:
            print(f"   Sections: {len(paper['sections'])}")
            print(f"   {'─'*40}")
            for section in paper['sections'][:5]:  # First 5 sections
                print(f"   • {section}")
            if len(paper['sections']) > 5:
                print(f"   • ... and {len(paper['sections']) - 5} more")

        # Preview abstract if available
        if 'content' in paper and 'abstract' in paper['content']:
            abstract = paper['content']['abstract']
            print(f"\n   ABSTRACT PREVIEW:")
            print(f"   {'─'*40}")
            lines = abstract.split('\n')[:4]
            for line in lines:
                if line.strip():
                    print(f"   {line}")
            if len(abstract) > 500:
                print(f"   ... (full abstract: {len(abstract)} characters)")

    # 8. EXECUTIVE SUMMARY
    print(f"\n⭐ EXECUTIVE SUMMARY")
    print(f"   {'─'*40}")

    # Create a quick summary
    summary_parts = []
    if topic_info:
        summary_parts.append(f"Research on '{topic_info.get('title', 'the topic')}'")
    summary_parts.append(f"analyzed {len(sources)} sources")
    summary_parts.append(f"identified {len(findings)} key findings")

    if limitations:
        summary_parts.append(f"noted {len(limitations)} limitations")

    if recommendations:
        summary_parts.append(f"proposed {len(recommendations)} recommendations")

    print(f"   This research {' and '.join(summary_parts)}.")

    if 'analysis' in result and 'confidence_score' in result['analysis']:
        conf = result['analysis']['confidence_score']
        confidence_level = "High" if conf > 0.7 else "Moderate" if conf > 0.5 else "Preliminary"
        print(f"   Overall confidence: {confidence_level} ({conf:.2f}/1.0)")

    print(f"\n" + "="*100)
    print("✅ RESEARCH COMPLETE")
    print("="*100)

In [36]:
# Run research and display immediately
async def run_and_display():
    topic = ResearchTopic(
        title="Discounted Cash Flow in modern world",
        domain="Finance",
        complexity="expert"
    )

    assistant = ResearchAssistantGraph()
    result = await assistant.run_research(topic)

    # Display the research
    display_research_improved(result)

    return result

# Execute
import asyncio
result = asyncio.run(run_and_display())

/tmp/ipython-input-2640601077.py:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  self.log_activity("coordination_started", {"topic": state["topic"].dict()})
INFO:research_coordinator:{"timestamp": "2026-01-27T21:49:33.224708", "agent": "research_coordinator", "activity": "coordination_started", "metadata": {"topic": {"title": "Discounted Cash Flow in modern world", "domain": "Finance", "complexity": "expert", "subtopics": []}}}
INFO:research_coordinator:{"timestamp": "2026-01-27T21:49:37.884652", "agent": "research_coordinator", "activity": "coordination_completed", "metadata": {"research_plan": "Certainly! Below is a comprehensive research plan for the topic **\"Discounted Cash Flow in Modern World\"** within the domain of Finance, designed at an expert level.\n\n---\n\n## \ud83d\udd0d Research Plan: Discounted Ca

Step: research_coordinator


INFO:search_specialist:{"timestamp": "2026-01-27T21:49:39.103675", "agent": "search_specialist", "activity": "search_completed", "metadata": {"queries": ["Discounted Cash Flow in modern world recent developments", "Finance Discounted Cash Flow in modern world research papers", "Discounted Cash Flow in modern world methodology best practices"], "results_found": 3, "citations_generated": 3}}
/tmp/ipython-input-2640601077.py:163: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  "topic": topic.dict(),


Step: search_specialist


INFO:analyst:{"timestamp": "2026-01-27T21:49:42.431962", "agent": "analyst", "activity": "analysis_completed", "metadata": {"topic": "Discounted Cash Flow in modern world", "sources_analyzed": 3, "confidence": 0.9}}


Step: analyst
Step: validator
Step: synthesizer
Step: writer

🎯 ADVANCED RESEARCH REPORT

📋 RESEARCH OVERVIEW
   ────────────────────────────────────────
   Topic data not found in result
   🔹 Research Phase: completed
   🔹 Sources Analyzed: 3
   🔹 Key Findings: 5

📝 RESEARCH PLAN
   ────────────────────────────────────────
   ### Analysis of Discounted Cash Flow in the Modern World
   #### 1. **Key Trends and Patterns**
   - **Increased Emphasis on Quantitative Valuation**  
   The collected data indicates a persistent demand for Disrupted Cash Flow (DCF) methods as core tools in determining a company’s or investment’s intrinsic value. Both historical and contemporary sources emphasize the significance of projecting future cash flows and appropriately discounting them to present value—a central pillar of DCF analysis.
   - **Integration of Quantitative Rigor and Qualitative Influence**  
   The responses consistently underscore the necessity of balancing quantitative rigor with qualit